In [ ]:
# ! pip install langchain tiktoken pypdf chromadb
# ! pip install transformers sentence_transformers==2.2.2
# ! pip install accelerate 
# ! pip install plum-dispatch==1.7.4 aspose.words Spire.PDF
# # bitsandbytes xformers

: 

In [1]:
import torch
torch.cuda.is_available()

True

In [1]:
# Loading necessary modeules
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import  HuggingFaceBgeEmbeddings
import aspose.words as aw
from spire.pdf.common import *
from spire.pdf import *
import os
import pandas as pd
import numpy as np


In [4]:
# Loading Pdf
dir_name = "bw_pdf"
file_loader = DirectoryLoader(dir_name, glob="./*.pdf", loader_cls=PyPDFLoader)
documents = file_loader.load()

In [5]:
# Removing New line Characters
for doc in documents:
    doc.page_content = doc.page_content.replace("\n", "")

In [6]:
# intializing TextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=300)

In [7]:
# splitting Documents to chunks
texts = text_splitter.split_documents(documents)

In [8]:
# Function Extract Table from document and return prompt converted to document
from langchain.docstore.document import Document
def extract_dataframe_list(file_path, temp_dir):
    page = []
    try:
        file_extention = file_path[file_path.rindex('.'):]
        df_list = []
        if (file_extention in [".xlsx", ".csv", ".html", ".pdf"]):
            if (file_extention == ".html"):
                df_list = pd.read_html(file_path)
            elif (file_extention == ".csv"):
                df_list.append(pd.read_csv(file_path))
            elif (file_extention == ".xlsx"):
                df_list.append(pd.read_excel(file_path))
            elif (file_extention == ".pdf"):
                pdf = PdfDocument()
                pdf.LoadFromFile(file_path)
                total_pages = pdf.Pages.Count
                pdf.Split('/kaggle/working/temp_file_{0}.pdf')
                for i in range(total_pages):
                    temp_html_name = f'/kaggle/working/temp_file_{i}.html'
                    temp_pdf_name = f'/kaggle/working/temp_file_{i}.pdf'

                    doc = aw.Document(temp_pdf_name)
                    os.remove(temp_pdf_name)
                    doc.save(temp_html_name)

                    try:
                        ls = pd.read_html(temp_html_name)
                        os.remove(temp_html_name)
                        df_list.extend(ls)
                        for j in range(len(ls)):
                            page.append(i+1)
                    except:
                        print('-', end='')
                for filename in os.listdir(temp_dir):
                    if os.path.isfile(os.path.join(temp_dir, filename)):
                        os.remove(os.path.join(temp_dir, filename))

        return df_list, page
    except:
        return [], page


def is_incremental(s1, s2):
    s1_ascii, s2_ascii = sum([ord(ele)
                             for ele in s1]), sum([ord(ele) for ele in s2])
    if (s1_ascii+1 == s2_ascii or s1_ascii == s2_ascii):
        return True
    else:
        return False


def check_incremental_row(tab_1, tab_2):
    # If first column incremental
    tab_1_last = tab_1.iloc[-1, 0]
    tab_2_first = tab_2.iloc[0, 0]
    tab_2_1st_col_name = tab_2.columns[0]

    # IF TAB1(1,LAST) == TAB2(1,1)
    if (is_incremental(str(tab_1_last), str(tab_2_first))):
        tab_1.loc[len(tab_1)] = tab_2.values[0]
        tab_2.columns = tab_1.columns
        return True
    # IF TAB1(1,LAST) == COL_NAME(1)
    elif (is_incremental(str(tab_1_last), str(tab_2_1st_col_name))):
        tab_2.columns
        tab_1.loc[len(tab_1)] = tab_2.columns
        tab_2.columns = tab_1.columns

        return True
    else:
        return False


def merge_same_column(df):
    i = 1
    while i < len(df.columns):
        if (df.columns[i-1] == df.columns[i]):
            for j in df.iterrows():
                if (j[1][0]) != j[1][1]:
                    val = f"{j[1][0]}"+f"{j[1][1]}"
                    j[1][0] = j[1][1] = val

            df = df.loc[:, ~df.columns.duplicated()].copy()

        else:

            same_in_both_column = 0
            for j in df.iterrows():
                try:
                    if (j[1, 0]) == j[1, 1]:
                        same_in_both_column += 1
                except:
                    continue
            if (same_in_both_column == df.shape[0]):
                val = f"{df.columns[i-1]}"+f"{df.columns[i]}"
                temp_dict = {df.columns[i-1]: val, df.columns[i]: val}
                df.rename(columns=temp_dict, inplace=True)
            else:
                i += 1
            df = df.loc[:, ~df.columns.duplicated()].copy()

    return df


def filter_table(df_list, page):
    # filter individual table
    filtered_pages = []
    clean_tables = []
    for i, df in enumerate(df_list):

        #         df = df.fillna("")
        merge_same_column(df)

        total_row = df.shape[0]
        total_column = df.shape[1]

        # to specify the column name if named [0,1,2,3...]
        if (df.columns == range(total_column)).all():
            updated_column_names = dict(zip(df.columns, df.iloc[0]))

            # Drop the first row since it's used for column renaming
            df = df.drop(0)
            df.reset_index(drop=True, inplace=True)
            total_row -= 1

            # Rename columns using the updated_column_names dictionary
            df.rename(columns=updated_column_names, inplace=True)

       # Delete rows if all elements are the same
        all_same_rows = df.apply(lambda row: row.nunique() != 1, axis=1)
        df = df[all_same_rows]

        # Delete columns if column name is null or if all elements are null
        df = df.dropna(axis=1, how='all')

        # delete columns if column name is nan
        df = df.drop(columns=[col for col in df.columns if col is np.nan])

        df = df.fillna("")
        df.index = range(len(df.index))
        try:
            if ((df.columns == [df.columns[0]]*len(df.columns)).all()):
                df.columns = df.iloc[0]
                df.drop(df.index[0], inplace=True)
                df.reset_index(drop=True, inplace=True)
            else:
                df = merge_same_column(df)
        except:
            print('error')
        filtered_pages.append(page[i])
        clean_tables.append(df)

    # merge tables
    i = 1
    while i < len(clean_tables):
        try:

            if clean_tables[i].shape[1] == clean_tables[i-1].shape[1]:
                # IF NO OF COLUMN SAME
                if ((clean_tables[i].columns == clean_tables[i-1].columns).all()):

                    # JOIN IF COLUMN NAME SAME
                    clean_tables[i -
                                 1] = pd.concat([clean_tables[i-1], clean_tables[i]])
                    clean_tables.pop(i)
                    filtered_pages.remove(filtered_pages[i])

                elif (check_incremental_row(clean_tables[i-1], clean_tables[i])):
                    clean_tables[i -
                                 1] = pd.concat([clean_tables[i-1], clean_tables[i]])
                    clean_tables.pop(i)
                    filtered_pages.remove(filtered_pages[i])
                else:
                    i += 1

            else:
                i += 1
        except:
            i += 1
#     fcl = []
#     for df in clean_tables:
#         # delete single row column table
# #         if (df.shape[0] <= 1 or df.shape[1] <=1):
# #             continue
# # #         elif()
# #         else:
#         fcl.append(df)

    return clean_tables, filtered_pages


def give_prompt_from_document(file_path):
    extracted_dataframes, pages = extract_dataframe_list(
        file_path, '/kaggle/working/')

    filtered_tables, filtered_pages = filter_table(extracted_dataframes, pages)
#     print(len(filtered_tables), len(filtered_pages))

    doc_list = []
    for x, df in enumerate(filtered_tables):
        df.reset_index(drop=True, inplace=True)

        des_list = []
        for i in range(df.shape[0]):
            description = f"For {i} "
            row_values = [
                f"{df.columns[j]} is {df.iloc[i,j]}" for j in range(df.shape[1])]
            null_element_in_row = sum(1 for value in row_values if value == "")
            description += ", ".join(row_values) + "."
            des_list.append(description)
        doc_text = ''
        for i, des in enumerate(des_list):
            if len(doc_text+des) < 1200:
                doc_text += des
                if i == len(des_list)-1:
                    doc = Document(page_content=doc_text, metadata={
                                   "source": file_path, "page": filtered_pages[x]})
                    doc_list.append(doc)
            else:
                doc = Document(page_content=doc_text, metadata={
                               "source": file_path, "page": filtered_pages[x]})
                doc_list.append(doc)
                doc_text = des

#         prompt = "\n\n".join(des_list)
#         doc =  Document(page_content=prompt, metadata={"source": file_path, "page": filtered_pages[x]})
#         doc_list.append(doc)
#         final_prompt += "\n\n" + prompt

    return doc_list, filtered_tables

In [10]:
from tqdm import tqdm
doc_list = []
for filename in tqdm(os.listdir(dir_name)):
    if os.path.isfile(os.path.join(dir_name, filename)):
        print(os.path.join(dir_name, filename))
        doc_list.extend(give_prompt_from_document(
            os.path.join(dir_name, filename))[0])
texts.extend(doc_list)

  0%|          | 0/19 [00:00<?, ?it/s]

------------------------------------------------------------------------------------

  5%|▌         | 1/19 [00:46<13:53, 46.31s/it]

--------------------------------

 11%|█         | 2/19 [00:58<07:29, 26.44s/it]

...

--------------------------

 58%|█████▊    | 11/19 [10:09<03:20, 25.02s/it]

-------

 63%|██████▎   | 12/19 [10:14<02:11, 18.76s/it]

-------------------------------------------------------------------------------------

In [3]:
model_name = "BAAI/bge-large-en-v1.5"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device': 'cuda'}
# set True to compute cosine similarity
encode_kwargs = {'normalize_embeddings': True}
bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]